In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
BASE_DIR = '/content/drive/MyDrive/KWS_project'



In [ ]:
!wget https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz
!tar -xzf speech_commands_v0.02.tar.gz -C /content/


--2025-11-25 08:18:23--  https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.207, 142.251.2.207, 74.125.137.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2428923189 (2.3G) [application/gzip]
Saving to: ‘speech_commands_v0.02.tar.gz’

speech_commands_v0. 100%[===================>]   2.26G  92.6MB/s    in 15s     

2025-11-25 08:18:38 (154 MB/s) - ‘speech_commands_v0.02.tar.gz’ saved [2428923189/2428923189]



In [ ]:
!ls /content



_background_noise_  forward  off			   testing_list.txt
backward	    four     on				   three
bed		    go	     one			   tree
bird		    happy    README.md			   two
cat		    house    right			   up
dog		    learn    sample_data		   validation_list.txt
down		    left     seven			   visual
drive		    LICENSE  sheila			   wow
eight		    marvin   six			   yes
five		    nine     speech_commands_v0.02.tar.gz  zero
follow		    no	     stop


In [ ]:
!python /content/drive/MyDrive/KWS_project/train_cnn_m5.py


Device: cuda
Base dir: /content/drive/MyDrive/KWS_project
Debug mode: False
Epochs: 10
Preparing datasets (this may download first time)...
100% 2.26G/2.26G [00:15<00:00, 162MB/s]
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding

In [ ]:
!python /content/drive/MyDrive/KWS_project/preprocess.py \
    --raw /content \
    --out /content/drive/MyDrive/KWS_project/processed \
    --sr 16000 \
    --n_mfcc 12



Found 6 background noise files.
Classes found: ['.config', 'backward', 'bed', 'bird', 'cat', 'dog', 'down', 'drive', 'eight', 'five', 'follow', 'forward', 'four', 'go', 'happy', 'house', 'learn', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'sample_data', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'visual', 'wow', 'yes', 'zero']
Processing classes:   0% 0/38 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1664
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=853
  warnings.warn(
Processing classes:   5% 2/38 [01:07<20:20, 33.91s/it]/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=955
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/librosa/core/spectrum.py:2

In [ ]:
!python /content/drive/MyDrive/KWS_project/train_lstm_plain.py \
    --meta /content/drive/MyDrive/KWS_project/processed/metadata.csv \
    --outdir /content/drive/MyDrive/KWS_project \
    --epochs 25 \
    --batch 64 \
    --lr 0.001



Labels: ['backward', 'bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'follow', 'forward', 'four', 'go', 'happy', 'house', 'learn', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'silence', 'six', 'stop', 'three', 'tree', 'two', 'up', 'visual', 'wow', 'yes', 'zero']
84872 9980 11005
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(

EPOCH 1/25
Train  acc: 0.676
Val    acc: 0.819
Test   acc: 0.810

EPOCH 2/25
Train  acc: 0.831
Val    acc: 0.853
Test   acc: 0.840

EPOCH 3/25
Train  acc: 0.854
Val    acc: 0.861
Test   acc: 0.854

EPOCH 4/25
Train  acc: 0.867
Val    acc: 0.868
Test   acc: 0.862

EPOCH 5/25
Train  acc: 0.875
Val    acc: 0.870
Test   acc: 0.865

EPOCH 6/25
Train  acc: 0.894
Val    acc: 0.882
Test   acc: 0.875

EPOCH 7/25
Train  acc: 0.9

In [ ]:
%%writefile /content/drive/MyDrive/KWS_project/eval_lstm_plain.py
import os
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from train_lstm_plain import MFCCDataset, collate_fn, PlainLSTM


def evaluate(model, loader, device):
    model.eval()
    total, correct = 0, 0

    with torch.no_grad():
        for x, lengths, labels in tqdm(loader, desc="Testing", leave=False):
            x, lengths, labels = x.to(device), lengths.to(device), labels.to(device)
            logits = model(x, lengths)
            preds = logits.argmax(1)

            total += labels.size(0)
            correct += (preds == labels).sum().item()

    return correct / total


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--meta", required=True)
    parser.add_argument("--weights", required=True)
    parser.add_argument("--outdir", default="/content/drive/MyDrive/KWS_project")
    args = parser.parse_args()

    df = pd.read_csv(args.meta)
    labels = sorted(df["label"].unique())

    df_test = df[df.split == "testing"]

    test_ds = MFCCDataset(df_test, labels)
    test_loader = DataLoader(test_ds, batch_size=64, shuffle=False, collate_fn=collate_fn)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Build model
    example = np.load(df_test.iloc[0]["path"])
    input_dim = example.shape[0]  # =36
    num_classes = len(labels)

    model = PlainLSTM(input_dim, 128, num_classes).to(device)
    model.load_state_dict(torch.load(args.weights, map_location=device))

    # Run test accuracy
    acc = evaluate(model, test_loader, device)
    print(f"\nFinal LSTM Test Accuracy: {acc:.4f}")

    # Append to results.csv
    results_path = os.path.join(args.outdir, "results.csv")

    if not os.path.exists(results_path):
        pd.DataFrame(columns=["Model", "Accuracy"]).to_csv(results_path, index=False)

    dfres = pd.read_csv(results_path)
    dfres.loc[len(dfres)] = ["LSTM-Plain", acc]
    dfres.to_csv(results_path, index=False)

    print("Updated results.csv")


if __name__ == "__main__":
    main()


Overwriting /content/drive/MyDrive/KWS_project/eval_lstm_plain.py


In [ ]:
!python /content/drive/MyDrive/KWS_project/eval_lstm_plain.py \
    --meta /content/drive/MyDrive/KWS_project/processed/metadata.csv \
    --weights /content/drive/MyDrive/KWS_project/lstm_plain_best.pth



/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
                                              
Final LSTM Test Accuracy: 0.8822
Updated results.csv


In [ ]:
!python /content/drive/MyDrive/KWS_project/train_cnn_m5_aug.py --augment_train --augment_valtest --augment_prob 0.10


Device: cuda
Base dir: /content/drive/MyDrive/KWS_project
Debug mode: False
Epochs: 10
Augment: True Augment prob: 0.1
Found noise files: 6
Preparing datasets (this may download first time)...
Labels (n): 35
['backward', 'bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'follow', 'forward', 'four', 'go', 'happy', 'house', 'learn', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'visual', 'wow', 'yes', 'zero']
Train E1: 100% 332/332 [03:42<00:00,  1.49it/s, acc=47.9, loss=1.8]
Eval Val: 100% 39/39 [00:27<00:00,  1.43it/s]
Val Accuracy: 0.6562 (6550/9981)
Saved checkpoint: /content/drive/MyDrive/KWS_project/cnn_m5_checkpoint.pth
Train E2: 100% 332/332 [04:05<00:00,  1.35it/s, acc=71, loss=0.987]
Eval Val: 100% 39/39 [00:28<00:00,  1.39it/s]
Val Accuracy: 0.6801 (6788/9981)
Saved checkpoint: /content/drive/MyDrive/KWS_project/cnn_m5_checkpoint.pth
Train E3: 100% 332/332 [03:45<00:00,  1.47it/s, acc=75.8, loss